In [1]:
import os
import glob
import numpy as np
import xarray as xr
import dask
import time
import itertools

import sys
p = os.path.abspath('/global/homes/q/qnicolas/')
if p not in sys.path:
    sys.path.append(p)

from tools.e5tools import *
xr.set_options(display_style='text') 
from orographicPrecipitation.observationsAndERA5.mountainUtils import BL_plevs
from orographicPrecipitation.precip_model_functions import humidsat,qsat

from mountainUtils import *

In [2]:
import dask
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status"
from dask.distributed import Client
client=Client()

/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40499 instead
  http_address["port"], self.http_server.port


In [3]:
MAINDIR = "/global/cscratch1/sd/qnicolas/regionsDataBig/"

Ghats      = MountainRangeCustom("ghats"      ,[64,82,7,24]   ,"Western Ghats"            ,70 ,[6,7,8]  ,[ 75  ,   9  ,  71.5, 20   ,2  ],path=MAINDIR)
Myanmar    = MountainRangeCustom("myanmar"    ,[84,102,7,24]  ,"Myanmar"                  ,60 ,[6,7,8]  ,[ 98  ,  11  ,  90  , 21   ,4  ],path=MAINDIR)
Vietnam    = MountainRangeCustom("vietnam"    ,[100,118,8,25] ,"Annamite range (Vietnam)" ,240,[10,11]  ,[108  ,  18  , 111  , 13   ,2  ],path=MAINDIR)
Malaysia   = MountainRangeCustom("malaysia"   ,[96,114,-2,15] ,"Malaysia"                 ,225,[11,12]  ,[102.5,   8  , 105.5,  3   ,2  ],path=MAINDIR)
Philippines= MountainRangeCustom("philippines",[117,135,5,22] ,"Philippines"              ,225,[11,12]  ,[123.5,  17  , 127  ,  9   ,2  ],path=MAINDIR)
Newbritain = MountainRangeCustom("newbritain" ,[145,163,-17,0],"New Britain"              ,320,[6,7,8]  ,[154  , - 5.5, 150  , -8.5 ,2  ],path=MAINDIR)
SMO        = MountainRangeCustom("smo"        ,[240,270,15,35],"Sierra Madre Occidental"  ,60 ,[7,8,9]  ,[254, 21, 249, 27.5, 3],path = "/global/cscratch1/sd/qnicolas/regionsData/")
Cascades   = MountainRangeCustom("cascades"   ,[222,240,43,60],"British Columbia cascades",45 ,[1,11,12],[235,47,227,54,3])

MRs = (Ghats, Myanmar    ,Vietnam    ,Malaysia   ,Philippines,Newbritain )   #SMO,)#Cascades,)#


#for MR in MRs:
#    print(MR.name)
#    MR.set_4dvar('T','128_130_t')
#    MR.set_4dvar('U','128_131_u')
#    MR.set_4dvar('V','128_132_v')
#    MR.set_4dvar('Q','128_133_q')
#    MR.set_4dvar('W','128_135_w')

In [4]:
Ghats      .box_computations = Ghats      .box
Myanmar    .box_computations = Myanmar    .box
Vietnam    .box_computations = Vietnam    .box
Malaysia   .box_computations = Malaysia   .box
Philippines.box_computations = Philippines.box
Newbritain .box_computations = Newbritain .box
SMO .box_computations = SMO .box
Cascades .box_computations = Cascades .box

allyears=range(2001,2021)

In [5]:
def resample_daily(ds):
    """performs the same job as resample(time="1D") but keeps missing days out"""
    ds.coords['time'] = ds.time.dt.floor('1D')
    return ds.groupby('time').mean()

# Loading daily data into zarr groups

## ERA5 plevs

In [6]:
def createzarr_era5_pl(MR,m,varcode):
    name = MR.name
    box = MR.box_computations
    years=allyears
    for y in years:
        filepath = MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name)
        if os.path.isdir(filepath):
            print("Already exists! - %s"%filepath)
            continue
        print("Doesn't exist! - %s"%filepath)
        ti=time.time()
        var = xr.open_mfdataset(ERA5D_PATH+"e5.oper.an.pl/{}{:02}/e5.oper.an.pl.{}.*.nc".format(y,m,varcode))
        sel_box(var,box).to_zarr(filepath,mode="w")
        print("%is"%(time.time()-ti))
        

In [7]:
for MR in MRs:
    months=MR.months
    for varcode in ["128_060_pv","128_130_t","128_131_u","128_132_v","128_133_q","128_135_w"]:
        for m in months:
            %time createzarr_era5_pl(MR,m,varcode)

Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200101.cascades.zarr
110s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200201.cascades.zarr
106s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200301.cascades.zarr
113s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200401.cascades.zarr
108s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200501.cascades.zarr
113s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200601.cascades.zarr
112s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200701.cascades.zarr
118s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200801.cascades.zarr
112s
Doesn't exist! - /global/cscratc

## ERA5 sfc - TRMM - GPM

In [9]:
def createzarr_era5_sfc(MR,m,varcode):
    name=MR.name
    box=MR.box_computations
    years=allyears
    filepath = MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)
    if os.path.isdir(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.sfc/*/e5.oper.an.sfc.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
    sel_box(var,box).to_zarr(filepath,mode="w")
        
def createzarr_trmm(MR,m):
    name=MR.name
    box=MR.box_computations
    years=range(2010,2014)
    var = xr.open_mfdataset([glob.glob(CMIP6_FOLDER+"obs4mip/NASA-GSFC/TRMM/observations/atmos/pr/3hr/NASA-GSFC/TRMM/*/*_{}{:02}*.nc".format(y,m))[0] for y in years])
    var = var.rename({'lat':'latitude','lon':'longitude'})
    var = var.reindex(latitude=list(reversed(var.latitude)))
    sel_box(var,box).to_zarr(MAINDIR + "trmm.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name),mode="w")

        

In [9]:
def flatten(t):
    return [item for sublist in t for item in sublist]
def createzarr_gpm(MR,m):
    name=MR.name
    box=MR.box_computations
    years=allyears
    filepath = MAINDIR + "gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name)
    if os.path.isdir(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob("/global/cscratch1/sd/pnicknis/gpm/daily/3B-DAY.MS.MRG.3IMERG.{}{:02}*-S000000-E235959.V06.nc4".format(y,m)) for y in years])
    var = var.rename({'lat':'latitude','lon':'longitude'})
    var = var.reindex(latitude=list(reversed(var.latitude)))
    var = var.assign_coords({'longitude': var.longitude%360})
    var = var.sortby('longitude')
    var['time'] = var.indexes['time'].to_datetimeindex()
    sel_box(var,box).to_zarr(filepath,mode="w")


In [10]:
def createzarr_era5_vinteg(name,m,varcode):
    name=MR.name
    box=MR.box_computations
    years=allyears
    filepath = MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)
    if os.path.isfile(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.vinteg/*/e5.oper.an.vinteg.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
    sel_box(var,box).to_zarr(filepath,mode="w")
        

In [11]:
def createzarr_era5_vinteg(MR,m,varcode):
    name=MR.name
    box=MR.box_computations
    years=allyears
    filepath = MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)
    if os.path.isfile(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.vinteg/*/e5.oper.an.vinteg.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
    sel_box(var,box).to_zarr(filepath,mode="w")
        

## To be run

In [10]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    for MR in MRs:
        print(MR.name)
        months=MR.months
        varcode = '128_034_sstk'
        for m in months:
            print(m)
            %time createzarr_era5_sfc(MR,m,varcode)

ghats
6
CPU times: user 3.25 s, sys: 1.83 s, total: 5.08 s
Wall time: 39.1 s
7
CPU times: user 3.45 s, sys: 1.66 s, total: 5.11 s
Wall time: 33.9 s
8
CPU times: user 3.77 s, sys: 1.67 s, total: 5.44 s
Wall time: 40.2 s
myanmar
6
CPU times: user 2.96 s, sys: 1.38 s, total: 4.35 s
Wall time: 27.4 s
7
CPU times: user 3.21 s, sys: 1.52 s, total: 4.73 s
Wall time: 29 s
8
CPU times: user 3.3 s, sys: 1.59 s, total: 4.89 s
Wall time: 30.7 s
vietnam
10
CPU times: user 2.75 s, sys: 1.48 s, total: 4.23 s
Wall time: 24.1 s
11
CPU times: user 2.83 s, sys: 1.45 s, total: 4.28 s
Wall time: 22 s
malaysia
11
CPU times: user 3.14 s, sys: 1.34 s, total: 4.47 s
Wall time: 25.7 s
12
CPU times: user 2.92 s, sys: 1.35 s, total: 4.27 s
Wall time: 32.6 s
philippines
11
CPU times: user 3.38 s, sys: 1.36 s, total: 4.74 s
Wall time: 35.6 s
12
CPU times: user 3.21 s, sys: 1.54 s, total: 4.75 s
Wall time: 30.9 s
newbritain
6
CPU times: user 3.14 s, sys: 1.58 s, total: 4.72 s
Wall time: 25.7 s
7
CPU times: user 2.96

# Time and space mean

A lot of this stuff can be made faster by first concatenating the zarr datasets, then calculating means etc

## Daily

In [12]:
def compute_spatialmean_pl(MR,varcode,where='above'):
    years = allyears
    name=MR.name
    months= MR.months
    spatialmeans=[]
    for y,m in itertools.product(years,months):
        ds = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name))
        varname = list(ds.data_vars)[0]
        ds=ds[varname]
        if where=='above':
            mask_era5= tilted_rect(ds,*MR.box_tilted,reverse=False)
        elif where=='upstream':
            mask_era5= tilted_rect(ds,*MR.box_tilted,reverse=True)
        spatialmeans.append(spatial_mean(resample_daily(ds),box=None,mask=mask_era5))
    temp=xr.concat(spatialmeans,dim='time')
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}-{}.{}.{}.{}mean.nc".format(varcode,years[0],years[-1],monthstr,name,where)
    temp.to_netcdf(filepath) 

In [ ]:
for MR in MRs:
    print(MR.name)
    %time compute_spatialmean_pl(MR,'128_130_t',where='upstream')
    %time compute_spatialmean_pl(MR,'128_133_q',where='upstream')

In [11]:
def concat_dailymeans_sfc(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    var_months  = [resample_daily(var[varname]) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{}.{}.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months,dim='time').sortby('time').to_netcdf(filepath) 


def compute_upstream_spatialmean_sfc(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    mask_ghats_era5= tilted_rect(var_months[0],*MR.box_tilted,reverse=True)
    var_months_sm  = [spatial_mean(resample_daily(var[varname]),box=None,mask=mask_ghats_era5) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{}.{}.upstreammean.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 

def compute_spatialmean_gpm(MR):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name)) for m in months]
    mask_ghats_gpm= tilted_rect(var_months[0],*MR.box_tilted,reverse=False)
    var_months_sm  = [spatial_mean(var.precipitationCal,box=None,mask=mask_ghats_gpm) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "gpm_imerg_v06.{}-{}.{}.{}.abovemean.nc".format(years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 

def concat_dailymeans_gpm(MR):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name)) for m in months]
    var_months  = [var.precipitationCal for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "gpm_imerg_v06.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    xr.concat(var_months,dim='time').sortby('time').to_netcdf(filepath) 
    

In [12]:
for MR in MRs:
    print(MR.name)
    varcode = '128_034_sstk'
    %time concat_dailymeans_sfc(MR,varcode)


ghats
CPU times: user 18.9 s, sys: 576 ms, total: 19.4 s
Wall time: 25.9 s
myanmar
CPU times: user 18.7 s, sys: 854 ms, total: 19.5 s
Wall time: 25.8 s
vietnam
CPU times: user 13.1 s, sys: 645 ms, total: 13.7 s
Wall time: 17.8 s
malaysia
CPU times: user 11.9 s, sys: 618 ms, total: 12.5 s
Wall time: 17.1 s
philippines
CPU times: user 12.9 s, sys: 544 ms, total: 13.5 s
Wall time: 17.8 s
newbritain
CPU times: user 19.8 s, sys: 853 ms, total: 20.6 s
Wall time: 27 s


In [11]:
def compute_upstream_spatialmean_vinteg(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    mask_ghats_era5= tilted_rect(var_months[0],*MR.box_tilted,reverse=True)
    var_months_sm  = [spatial_mean(resample_daily(var[varname]),box=None,mask=mask_ghats_era5) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{}.{}.upstreammean.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 

def concat_dailymeans_vinteg(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    var_months  = [resample_daily(var[varname]) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{}.{}.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months,dim='time').sortby('time').to_netcdf(filepath) 


In [12]:
for MR in MRs:
    print(MR)
    concat_dailymeans_vinteg(MR,'162_071_viwve')
    concat_dailymeans_vinteg(MR,'162_072_viwvn')

In [12]:
def compute_ebeLeLstar(MR,kind='shallow'):
    years = allyears
    name=MR.name
    months= MR.months
    ebs=[]
    eLs=[]
    eLstars=[]
    if kind=='shallow':
        lft_top_pressure=700.
        lft_bot_pressure=900.
        suffix=''
    elif kind=='deep':
        lft_top_pressure=500.
        lft_bot_pressure=850. 
        suffix='deep'
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        eb = 2.5e6/1004*q.sel(level=slice(lft_bot_pressure,1100))+t.sel(level=slice(lft_bot_pressure,1100))
        tL = t.sel(level=slice(lft_top_pressure,lft_bot_pressure-1))
        qL = q.sel(level=slice(lft_top_pressure,lft_bot_pressure-1))
        _,qs,_=humidsat(tL,tL.level)
        eL = 2.5e6/1004*qL+tL
        eLstar = 2.5e6/1004*qs+tL
        
        ebs.append(resample_daily(eb))
        eLs.append(resample_daily(eL))
        eLstars.append(resample_daily(eLstar))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(ebs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eb{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLstars,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eLstar{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [13]:
def compute_tLqL(MR,kind='shallow'):
    years = allyears
    name=MR.name
    months= MR.months
    tLs=[]
    qLs=[]
    if kind=='shallow':
        lft_top_pressure=700.
        lft_bot_pressure=900.
        suffix=''
    elif kind=='deep':
        lft_top_pressure=500.
        lft_bot_pressure=850. 
        suffix='deep'
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        tL = t.sel(level=slice(lft_top_pressure,lft_bot_pressure-1)).mean('level')
        qL = q.sel(level=slice(lft_top_pressure,lft_bot_pressure-1)).mean('level')
        tLs.append(resample_daily(tL))
        qLs.append(resample_daily(qL))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(tLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.tL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(qLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.qL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [17]:
for MR in MRs:
    print(MR.name)
    #%time compute_ebeLeLstar(MR,'deep')
    #%time compute_ebeLeLstar(MR,'shallow')
    #%time compute_tLqL(MR,'deep')
    #%time compute_tLqL(MR,'shallow')
    %time compute_uLvL(MR)
    %time compute_u900v900(MR)

cascades
2001 1
2001 11
2001 12
2002 1
2002 11
2002 12
2003 1
2003 11
2003 12
2004 1
2004 11
2004 12
2005 1
2005 11
2005 12
2006 1
2006 11
2006 12
2007 1
2007 11
2007 12
2008 1
2008 11
2008 12
2009 1
2009 11
2009 12
2010 1
2010 11
2010 12
2011 1
2011 11
2011 12
2012 1
2012 11
2012 12
2013 1
2013 11
2013 12
2014 1
2014 11
2014 12
2015 1
2015 11
2015 12
2016 1
2016 11
2016 12
2017 1
2017 11
2017 12
2018 1
2018 11
2018 12
2019 1
2019 11
2019 12
2020 1
2020 11
2020 12
CPU times: user 39.6 s, sys: 4.36 s, total: 44 s
Wall time: 1min 32s
2001 1
2001 11
2001 12
2002 1
2002 11
2002 12
2003 1
2003 11
2003 12
2004 1
2004 11
2004 12
2005 1
2005 11
2005 12
2006 1
2006 11
2006 12
2007 1
2007 11
2007 12
2008 1
2008 11
2008 12
2009 1
2009 11
2009 12
2010 1
2010 11
2010 12
2011 1
2011 11
2011 12
2012 1
2012 11
2012 12
2013 1
2013 11
2013 12
2014 1
2014 11
2014 12
2015 1
2015 11
2015 12
2016 1
2016 11
2016 12
2017 1
2017 11
2017 12
2018 1
2018 11
2018 12
2019 1
2019 11
2019 12
2020 1
2020 11
2020 12
CP

In [15]:
def compute_uLvL(MR):
    years = allyears
    name=MR.name
    months= MR.months
    uLs=[]
    vLs=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        u = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_131_u',y,m,name)).U
        v = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_132_v',y,m,name)).V
        uL = u.sel(level=slice(700.,899.)).mean('level')
        vL = v.sel(level=slice(700.,899.)).mean('level')
        uLs.append(resample_daily(uL))
        vLs.append(resample_daily(vL))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(uLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.uL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(vLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.vL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [16]:
def compute_u900v900(MR):
    years = allyears
    name=MR.name
    months= MR.months
    uLs=[]
    vLs=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        u = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_131_u',y,m,name)).U
        v = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_132_v',y,m,name)).V
        uL = u.sel(level = 900.)
        vL = v.sel(level = 900.)
        uLs.append(resample_daily(uL))
        vLs.append(resample_daily(vL))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(uLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.u900.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(vLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.v900.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

# Additional

In [13]:
def compute_ebeLeLstardeep(MR):
    years = allyears
    name=MR.name
    months= MR.months
    ebs=[]
    eLs=[]
    eLstars=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        eb = 2.5e6/1004*q.sel(level=slice(850,1100))+t.sel(level=slice(850,1100))
        tL = t.sel(level=slice(500.,849.))
        qL = q.sel(level=slice(500.,849.))
        _,qs,_=humidsat(tL,tL.level)
        eL = 2.5e6/1004*qL+tL
        eLstar = 2.5e6/1004*qs+tL
        
        ebs.append(resample_daily(eb))
        eLs.append(resample_daily(eL))
        eLstars.append(resample_daily(eLstar))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(ebs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.ebdeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eLdeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLstars,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eLstardeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 

def compute_thetaebthetaeLthetaeLstar(MR,kind='shallow'):
    years = allyears
    name=MR.name
    months= MR.months
    thetaebs=[]
    thetaeLs=[]
    thetaeLstars=[]
    if kind=='shallow':
        lft_top_pressure=700.
        lft_bot_pressure=900.
        suffix=''
    elif kind=='deep':
        lft_top_pressure=500.
        lft_bot_pressure=850. 
        suffix='deep'
    
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        thetaeb = thetaeb_vectorized(t,q,lft_bot_pressure,pname='level')
        thetaeL = thetaeL_vectorized(t,q,lft_top_pressure,lft_bot_pressure,pname='level')
        thetaeLstar = thetaeLstar_vectorized(t,lft_top_pressure,lft_bot_pressure,pname='level')
        thetaebs.append(resample_daily(thetaeb).load())
        thetaeLs.append(resample_daily(thetaeL).load())
        thetaeLstars.append(resample_daily(thetaeLstar))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
        
    temp=xr.concat(thetaebs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.thetaeb{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(thetaeLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.thetaeL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(thetaeLstars,dim='time')
    filepath = MAINDIR + "e5.diagnostic.thetaeLstar{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 

In [ ]:
for MR in MRs:
    print(MR.name)
    #%time compute_thetaebthetaeLthetaeLstar(MR,'shallow')
    %time compute_thetaebthetaeLthetaeLstar(MR,'deep')

# Calculating clean $\theta_{eB}$ and the rest

In [7]:
era5plevs = 100.*np.array([1., 2., 3., 5., 7., 10., 20., 30., 50., 70.,
        100., 125., 150., 175., 200., 225., 250., 300., 350., 400.,
        450., 500., 550., 600., 650., 700., 750., 775., 800., 825.,
        850., 875., 900., 925., 950., 975., 1000.])
def p_k(ps,plevs=era5plevs,side='right'):
    return plevs[np.maximum(np.searchsorted(plevs,ps,side=side)-1,0)]
def p_kp1(ps,plevs=era5plevs,side='right'):
    return plevs[np.minimum(np.searchsorted(plevs,ps,side=side),len(plevs)-1)]

def integrate(y,x,x1,x2):
    "Integrate y*dx between x1 and x2, using trapezoidal rule. x1 has to be > min(x); x2 can be higher than max(x), in which case constant padding is used"
    assert x1>=x[0]
    y1 = np.interp(x1,x,y)
    y2 = np.interp(x2,x,y)
    
    idxs = (x>x1)*(x<x2)
    xi = np.concatenate(([x1],x[idxs],[x2]))
    yi = np.concatenate(([y1],y[idxs],[y2]))
    return np.trapz(yi,xi)
def pressure_avg_bounds(ds,pt,ps):
    """Compute a vertical average in pressure coordinates, between two pressure levels, with linear interpolation in between.
    For more clarity, this does the same as the 'integrate' function above, but in a vectorized way hence orders of mag faster"""
    pmin = ds.level[0]
    pmax = ds.level[-1]
    ds_vinteg_inner = (ds*(100*ds.level<=ps)*(100*ds.level>=pt)).integrate("level")*100
    
    pk = xr.apply_ufunc(p_k,ps,kwargs={'plevs':100*np.array(ds.level)})
    pkp1 = xr.apply_ufunc(p_kp1,ps,kwargs={'plevs':100*np.array(ds.level)})
    fk = ds.sel(level=pk/100)
    fkp1 = ds.sel(level=pkp1/100)
    correction = - fk/2 * (pkp1-pk) * (ps < 100*ds.level[-1])
    pkp1 += (pkp1==pk) # avoid dividing by zero at the next line
    ds_vinteg_below = (ps-pk)*fk + (fkp1-fk)*(ps-pk)**2/(pkp1-pk)/2 + correction
    
    pk = xr.apply_ufunc(p_k,pt,kwargs={'plevs':100*np.array(ds.level),'side':'left'})
    pkp1 = xr.apply_ufunc(p_kp1,pt,kwargs={'plevs':100*np.array(ds.level),'side':'left'})
    fk = ds.sel(level=pk/100)
    fkp1 = ds.sel(level=pkp1/100)
    correction = - fkp1/2 * (pkp1-pk) * (pt> 100*ds.level[0])
    pk -= (pkp1==pk) # avoid dividing by zero at the next line
    ds_vinteg_above = (pkp1-pt)*fk + (fkp1-fk)*(pkp1-pk)*(1-((pt-pk)/(pkp1-pk))**2)/2 + correction    
    
    return (ds_vinteg_inner + ds_vinteg_below + ds_vinteg_above)/(ps-pt)

def compute_thetaebthetaeLthetaeLstar(MR,kind='deep'):
    years = allyears
    name=MR.name
    months= MR.months
    monthstr = '-'.join(["{:02}".format(m) for m in months])

    if kind=='shallow':
        lft_top_pressure=700.
        lft_bot_pressure=900.
        suffix=''
    elif kind=='deep':
        lft_top_pressure=500.
        lft_bot_pressure=850. 
        suffix='deep'
    elif kind=='semi':
        lft_top_pressure=600.
        lft_bot_pressure=900. 
        suffix='semi'
        ###suffix='dbl'
        
    t_all = xr.open_dataset(MR.path + 'e5.oper.an.pl.128_130_t.ll025sc.{}-{}.{}.{}.nc'.format(years[0],years[-1],monthstr,name)).T.sortby('time').sel(level=slice(lft_top_pressure,1000.))
    q_all = xr.open_dataset(MR.path + 'e5.oper.an.pl.128_133_q.ll025sc.{}-{}.{}.{}.nc'.format(years[0],years[-1],monthstr,name)).Q.sortby('time').sel(level=slice(lft_top_pressure,1000.))
    
    pname='level'
    es,qs,_= humidsat(t_all,t_all[pname])
    thetae = thetae_perso(t_all,q_all,es,qs,pname=pname)
    thetaestar = thetaestar_perso(t_all,es,qs,pname=pname)  
    
    # thetaeb = pressure_avg_bounds(thetae,MR.vars['SP']-1e4,MR.vars['SP'])
    # thetaeL = pressure_avg_bounds(thetae,lft_top_pressure*100,MR.vars['SP']-1e4)
    # thetaeLstar = pressure_avg_bounds(thetaestar,lft_top_pressure*100,MR.vars['SP']-1e4)
    #######thetaeb = pressure_avg_bounds(thetae,lft_bot_pressure*100,MR.vars['SP'])*np.sqrt(-1+2*(MR.vars['SP']>lft_bot_pressure*100))
    thetaeL = thetae.sel(level=slice(lft_top_pressure,lft_bot_pressure)).integrate('level')/(lft_bot_pressure-lft_top_pressure)#equivalently pressure_avg_bounds(thetae,lft_top_pressure*100,lft_bot_pressure*100)
    thetaeLstar = thetaestar.sel(level=slice(lft_top_pressure,lft_bot_pressure)).integrate('level')/(lft_bot_pressure-lft_top_pressure)
    
    #######filepath = MR.path + "e5.diagnostic.thetaeb{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    #######thetaeb.to_netcdf(filepath) 
    
    filepath = MR.path + "e5.diagnostic.thetaeL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    thetaeL.to_netcdf(filepath) 
    
    filepath = MR.path + "e5.diagnostic.thetaeLstar{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    thetaeLstar.to_netcdf(filepath) 


In [8]:
era5sp = e5_climatology('128_134_sp',years=allyears)
for MR in MRs:
    MR.set_2dvar('SP',era5sp)

2.25201153755188


In [9]:
for MR in (Ghats,):
    print(MR.name)
    %time compute_thetaebthetaeLthetaeLstar(MR,'deep')

ghats
CPU times: user 1min 21s, sys: 43.9 s, total: 2min 5s
Wall time: 1min 55s


In [ ]:
#from time import sleep
#for MR in MRs:
#    years = allyears
#    name=MR.name
#    months= MR.months
#    monthstr = '-'.join(["{:02}".format(m) for m in months]) 
#    suffix='semi2'
#    filepath = MR.path + "e5.diagnostic.thetaeb{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
#    test = xr.open_dataarray(filepath)
#    test = test + np.sqrt(-1*np.isinf(test))
#    os.remove(filepath)
#    sleep(1)
#    test.to_netcdf(filepath)

In [9]:
def compute_tLqL(MR,kind='deep'):
    years = allyears
    name=MR.name
    months= MR.months
    monthstr = '-'.join(["{:02}".format(m) for m in months])

    if kind=='shallow':
        lft_top_pressure=700.
        lft_bot_pressure=900.
        suffix=''
    elif kind=='deep':
        lft_top_pressure=500.
        lft_bot_pressure=850. 
        suffix='deep'
    elif kind=='semi':
        lft_top_pressure=600.
        lft_bot_pressure=900. 
        #######suffix='semi'
        suffix='dbl'
        
    t_all = xr.open_dataset(MR.path + 'e5.oper.an.pl.128_130_t.ll025sc.{}-{}.{}.{}.nc'.format(years[0],years[-1],monthstr,name)).T.sortby('time').sel(level=slice(lft_top_pressure,lft_bot_pressure))
    q_all = xr.open_dataset(MR.path + 'e5.oper.an.pl.128_133_q.ll025sc.{}-{}.{}.{}.nc'.format(years[0],years[-1],monthstr,name)).Q.sortby('time').sel(level=slice(lft_top_pressure,lft_bot_pressure))
    tL = pressure_avg_bounds(t_all,lft_top_pressure*100,MR.vars['SP']-1e4)
    qL = pressure_avg_bounds(q_all,lft_top_pressure*100,MR.vars['SP']-1e4)
    #tL = t_all.integrate('level')/(lft_bot_pressure-lft_top_pressure)
    #qL = q_all.integrate('level')/(lft_bot_pressure-lft_top_pressure)
    
    filepath = MR.path + "e5.diagnostic.tL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    tL.to_netcdf(filepath) 
    
    filepath = MR.path + "e5.diagnostic.qL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    qL.to_netcdf(filepath) 
    

In [10]:
for MR in MRs:
    print(MR.name)
    %time compute_tLqL(MR,'semi')

ghats
CPU times: user 42.6 s, sys: 22 s, total: 1min 4s
Wall time: 1min 18s
myanmar
CPU times: user 43.7 s, sys: 22.4 s, total: 1min 6s
Wall time: 1min 27s
vietnam
CPU times: user 26.4 s, sys: 14.2 s, total: 40.6 s
Wall time: 39 s
malaysia
CPU times: user 27 s, sys: 13.3 s, total: 40.4 s
Wall time: 43.6 s
philippines
CPU times: user 26.2 s, sys: 12.6 s, total: 38.9 s
Wall time: 37.8 s
newbritain
CPU times: user 41.1 s, sys: 20.1 s, total: 1min 1s
Wall time: 1min


In [10]:
def compute_uBLvBL(MR):
    years = allyears
    name=MR.name
    months= MR.months
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    lft_top_pressure=600.
    lft_bot_pressure=900. 
    u_all = xr.open_dataset(MR.path + 'e5.oper.an.pl.128_131_u.ll025sc.{}-{}.{}.{}.nc'.format(years[0],years[-1],monthstr,name)).U.sortby('time').sel(level=slice(lft_top_pressure,1100))
    v_all = xr.open_dataset(MR.path + 'e5.oper.an.pl.128_132_v.ll025sc.{}-{}.{}.{}.nc'.format(years[0],years[-1],monthstr,name)).V.sortby('time').sel(level=slice(lft_top_pressure,1100))
    #uL = pressure_avg_bounds(u_all,MR.vars['SP']-1e4,MR.vars['SP'])
    #vL = pressure_avg_bounds(v_all,MR.vars['SP']-1e4,MR.vars['SP'])
    uL = pressure_avg_bounds(u_all,lft_bot_pressure,MR.vars['SP'])
    vL = pressure_avg_bounds(v_all,lft_bot_pressure,MR.vars['SP'])
        
    #tL = t_all.integrate('level')/(lft_bot_pressure-lft_top_pressure)
    #qL = q_all.integrate('level')/(lft_bot_pressure-lft_top_pressure)
    
    filepath = MR.path + "e5.diagnostic.uBL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    uL.to_netcdf(filepath) 
    
    filepath = MR.path + "e5.diagnostic.vBL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    vL.to_netcdf(filepath) 
    

In [11]:
for MR in MRs:
    print(MR.name)
    %time compute_uBLvBL(MR)

ghats
CPU times: user 46.4 s, sys: 24.6 s, total: 1min 10s
Wall time: 2min 10s
myanmar
CPU times: user 50.9 s, sys: 28.4 s, total: 1min 19s
Wall time: 2min 20s
vietnam
CPU times: user 32.1 s, sys: 17.1 s, total: 49.1 s
Wall time: 1min 7s
malaysia
CPU times: user 30.2 s, sys: 15.8 s, total: 46 s
Wall time: 1min 1s
philippines
CPU times: user 28.9 s, sys: 15.9 s, total: 44.8 s
Wall time: 1min 3s
newbritain
CPU times: user 50.3 s, sys: 28 s, total: 1min 18s
Wall time: 2min 40s


In [10]:
def compute_wL(MR):
    years = allyears
    name=MR.name
    months= MR.months
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    lft_top_pressure=600.
    lft_bot_pressure=900. 
    w_all = xr.open_dataset(MR.path + 'e5.oper.an.pl.128_135_w.ll025sc.{}-{}.{}.{}.nc'.format(years[0],years[-1],monthstr,name)).W.sortby('time').sel(level=slice(lft_top_pressure,lft_bot_pressure)) 
    wL = w_all.integrate('level')/(lft_bot_pressure-lft_top_pressure)

    filepath = MR.path + "e5.diagnostic.wL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    wL.to_netcdf(filepath) 

In [11]:
for MR in MRs:
    print(MR.name)
    %time compute_wL(MR)

ghats
CPU times: user 1.93 s, sys: 1.93 s, total: 3.86 s
Wall time: 32 s
myanmar


FileNotFoundError: [Errno 2] No such file or directory: b'/global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_135_w.ll025sc.2001-2020.06-07-08.myanmar.nc'

vietnam


FileNotFoundError: [Errno 2] No such file or directory: b'/global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_135_w.ll025sc.2001-2020.10-11.vietnam.nc'

malaysia


FileNotFoundError: [Errno 2] No such file or directory: b'/global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_135_w.ll025sc.2001-2020.11-12.malaysia.nc'

philippines
CPU times: user 809 ms, sys: 1.14 s, total: 1.95 s
Wall time: 10.5 s
newbritain


FileNotFoundError: [Errno 2] No such file or directory: b'/global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_135_w.ll025sc.2001-2020.06-07-08.newbritain.nc'

# Concatenate pressure level data

In [6]:
def concat_dailymeans_pl(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name)) for m in months for y in years]
    varname = list(var_months[0].data_vars)[0]
    var_months  = [resample_daily(var[varname]) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}-{}.{}.{}.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months,dim='time').to_netcdf(filepath) 


In [8]:
for MR in MRs:
    print(MR.name)
    %time concat_dailymeans_pl(MR,'128_135_w')

ghats
CPU times: user 36.9 s, sys: 3.58 s, total: 40.4 s
Wall time: 56.8 s
myanmar


GroupNotFoundError: group not found at path ''

vietnam


GroupNotFoundError: group not found at path ''

malaysia


GroupNotFoundError: group not found at path ''

philippines
CPU times: user 23.6 s, sys: 2.39 s, total: 26 s
Wall time: 35.6 s
newbritain


GroupNotFoundError: group not found at path ''

# monthly data

In [8]:
gpm_all = xr.open_mfdataset("/global/cfs/projectdirs/m3310/qnicolas/GPM-IMERG/*.nc")

gpm_20012020 = gpm_all.sel(time=((gpm_all['time.year']>=2001)&(gpm_all['time.year']<=2020)))

In [11]:
gpm_20012020.groupby('time.month').mean().to_netcdf('/global/cscratch1/sd/qnicolas/temp/GPM-IMERG.monthly.2001-2020.nc')


In [12]:
u100 = xr.open_dataarray('/global/cscratch1/sd/qnicolas/temp/e5.monthly.228_246_100u.2001-2020.nc')
v100 = xr.open_dataarray('/global/cscratch1/sd/qnicolas/temp/e5.monthly.228_247_100v.2001-2020.nc')

u100.groupby('time.month').mean().to_netcdf('/global/cscratch1/sd/qnicolas/temp/e5.climatology.228_246_100u.2001-2020.nc')
v100.groupby('time.month').mean().to_netcdf('/global/cscratch1/sd/qnicolas/temp/e5.climatology.228_247_100v.2001-2020.nc')

